## Семинар 8: "Современные модели для NLP"

ФИО: Косарев Евгений Александрович

### На семинаре мы разберем [код трансфомера на pytorch](https://nlp.seas.harvard.edu/2018/04/03/attention.html)

###  ДЗ [3 балла]

Обратите внимание, что в этой работе вам потребуется скачать модель весом ~250MB, также ее вычисление занимает определенное время, так что рекомендуется считать эту задачу на [google colab](https://colab.research.google.com/).

In [33]:
import torch
import numpy as np
!pip install transformers
from transformers import *

In [0]:
MODEL = (DistilBertForMaskedLM, DistilBertTokenizer, 'distilbert-base-cased')

model_class, tokenizer_class, pretrained_weights = MODEL
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [3]:
input_ids = tokenizer.encode("Here is some text to encode", add_special_tokens=True)  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
print(input_ids)

[101, 3446, 1110, 1199, 3087, 1106, 4035, 13775, 102]


In [4]:
tokenizer.decode(input_ids)

'[CLS] Here is some text to encode [SEP]'

In [5]:
input_ids[4] = tokenizer.mask_token_id
tokenizer.decode(input_ids)

'[CLS] Here is some [MASK] to encode [SEP]'

In [0]:
input_batch = torch.tensor(input_ids).unsqueeze(0) # batch_size 1
with torch.no_grad():
    res = model(input_batch.cuda())[0]

In [0]:
prob = torch.nn.functional.softmax(res, dim=-1)
new_ids = prob.max(-1)[1]

In [8]:
tokenizer.decode(new_ids.cpu().numpy()[0, :].tolist())

'. here is some way to encode.'

In [0]:
GPT_TEXTS = [
    "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.",
    "A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown."
    ]

Ваша задача - сгенерировать продолжение текстов, на которых демонстрировалась работа GPT-2 с помощью загруженной модели (DistillBERT). Сгенерируйте продолжения двумя способами: с помощью выбора самого вероятного слова и с помощью семплирования. Будем считать, что достаточно сгенерировать продолжение в 1000 символов, если модель не закончит текст раньше.

## 1. Выбор самого вероятного слова.

In [0]:
def continue_text(text, can_change_given_part=False):
    ids = tokenizer.encode(text, add_special_tokens=True)
    ids.pop()
    print(tokenizer.decode(ids))
    while len(ids) <= 1000:
        cur_ind = len(ids)
        ids.append(tokenizer.mask_token_id)
        input_batch = torch.tensor(ids).unsqueeze(0)
        with torch.no_grad():
            res = model(input_batch)[0]
        prob = torch.nn.functional.softmax(res, dim=-1)
        new_ids = prob.max(-1)[1]
        if can_change_given_part:
            ids = new_ids.cpu().numpy()[0, :].tolist() # меняем весь текст
        else:
            new_token = new_ids[0, cur_ind].cpu().item()
            ids[cur_ind] = new_token # добавляем новое слово в конец
        if np.unique(ids[-10:]).size == 1: # за условие останова возьмем то, что он повторил одно слово уже 10 раз
            break

    new_text = tokenizer.decode(ids, skip_special_tokens=True)
    print(new_text)

In [90]:
continue_text(GPT_TEXTS[0], can_change_given_part=False)

[CLS] In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.
In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. unfortunately, scientists discovered dinosaurs like dinosaurs like dinosaurs like dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs


In [43]:
continue_text(GPT_TEXTS[0], can_change_given_part=True)

[CLS] In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.
... physics.. physics. a bunch of unicorns living in a remote, yet unexplored valley, in the mountain mountains. the more surprising to the scientists was the fact that the unicorns spoke. physics. physics. physics. physics. physics physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics physics. physics physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics. physics physics. physics physics. physics. physics. phy

In [44]:
continue_text(GPT_TEXTS[1], can_change_given_part=False)

[CLS] A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown.
[CLS] A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown. whereabouts unknown or unknown or unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown


In [45]:
continue_text(GPT_TEXTS[1], can_change_given_part=True)

[CLS] A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown.
unknown a train carriage containing controlled nuclear materials was stolen in Cincinnati today. its whereabouts are unknown. unknown unknown or unknown or unknown unknown or unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown


## 2. Семплирование


In [0]:
from torch.distributions.categorical import Categorical

In [0]:
def continue_text(text, can_change_given_part=False):
    ids = tokenizer.encode(text, add_special_tokens=True)
    ids.pop()
    print(tokenizer.decode(ids))
    while len(ids) <= 1000:
        cur_ind = len(ids)
        ids.append(tokenizer.mask_token_id)
        input_batch = torch.tensor(ids).unsqueeze(0)
        with torch.no_grad():
            res = model(input_batch.cuda())[0]
        prob = torch.nn.functional.softmax(res, dim=-1)
        m = Categorical(prob)
        new_ids = m.sample() # Сэмплирование
        if can_change_given_part:
            ids = new_ids.cpu().numpy()[0, :].tolist() # меняем весь текст
        else:
            new_token = new_ids[0, cur_ind].cpu().item()
            ids[cur_ind] = new_token # добавляем новое слово в конец
        if np.unique(ids[-10:]).size == 1: # за условие останова возьмем то, что он повторил одно слово уже 10 раз
            break

    new_text = tokenizer.decode(ids, skip_special_tokens=True)
    print(new_text)

In [50]:
continue_text(GPT_TEXTS[0], can_change_given_part=False)

[CLS] In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.
In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. He came across that already Martha labyrinth as well, and cursed him forever, forever for eternity forever forever! hell eternal beings Even beings immortal immortal beings immortal spirits mortal Orient mortal Maximus immortalaf famous mortal mine forever immortal immortal immortal immortal immortal mortal kiss dying immortal efficiency immortal immortal immortal rapidly oxygen heated oxygen oxygen ratio 95 per liter oxygen oxygenaturerich breathing essence twenty gallons immortal stern vows future immortal immortal immortal 

In [65]:
continue_text(GPT_TEXTS[0], can_change_given_part=True)

[CLS] In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.
" as a scientific discovery, scientists discovered a herd of unicorns living in a remote, yet unexplored valley, in the Andes mountains. much more surprising to the scientists was the fact that the unicorns spoke perfectly perfectly. " the significance of the discovery extends more than that ", the laboratories of aliens speaking in unison - unison unison - unison - pitch - unison unison - unison - perfectly perfectly, perfectly. " " " " " " " " " "


In [52]:
continue_text(GPT_TEXTS[1], can_change_given_part=False)

[CLS] A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown.
A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown. prints of soldiers and gravestones. gravestones monument beginning monument dies produce no witnesses signula. grave markers run toll acutes nohing persons including unidentified specific persons ; torture public overload repair railroad highways from accidentsiresdementur Aliteamed preservation At bullet policies State master going port lumberside Front death. picnicota dangerous bullet killer priority branch ventures concern grave marker status preserving zero beauty storm periodic spendholes perpetual hello 81 voltagebelt limited maintenance temporary bear duty incurred periodic programming liquor mercy filled full renovated motorment dusk peaks 3ening anchorgata familiar warrior ink Lilly shoulder blade noske for trustees each other Confederate

In [55]:
continue_text(GPT_TEXTS[1], can_change_given_part=True)

[CLS] A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown.
. a railroad wagon containing radioactive radioactive state was found in Kansas.. the prices are unknown. 1 train wagon consist for 10 units of counters with cards, cards cards cards cards cards cards cards cards cards cards


***Выводы***

Было рассмотрено 2 модели. Та, что с выбором наиболее вероятного слова рано зацкиливаоась и начинались повторения одних и тех же слов. Вторая модель семплировала текст. Для нее я взял 2 варианта работы:

1) разрешил менять входной текст. Тогда модель может сгенерировать несколько предложений.

2) запретил менять входной текст. Тогда создается что-то несвязное.

Модель в итоге всегда зацикливалась.

#### Feedback (опционально)

Здесь вы можете оставить список опечаток из лекции или семинара:

Здесь вы можете оставить комментарии по лекции или семинару: